In [33]:
# libraries
#!pip install rpy2
#!pip install pandas
#!pip install keras
#!pip install imblearn

    100% |████████████████████████████████| 92kB 660kB/s 
  Running setup.py bdist_wheel for imbalanced-learn ... - \ | / done
  Stored in directory: /Users/Kozodoi/Library/Caches/pip/wheels/b8/20/bd/0b775f7e5d413ac72562b1a5126598bcb6e0eae10da659be9f
Successfully built imbalanced-learn


In [1]:
# libraries
import rpy2.robjects as robjects
import pandas as pd
import keras
from keras.layers import Input, Dense, Embedding, concatenate, Flatten, Dropout, Reshape
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import Adam
from sklearn import metrics

Using TensorFlow backend.


## 1. DATA PREPARATION

In [2]:
# load the data
path = "/Users/Kozodoi/Documents/Competitions/DSG_2017/"
data = pd.read_csv(path + "data/data_flow.csv")
data.shape

(2339529, 35)

In [3]:
# add observation index
data["row_index"] = data.index

In [4]:
# data partitioning
tr = data.query("dataset == 'train'")
ts = data.query("dataset == 'test'")
kn = data.query("dataset != 'unknown'")
un = data.query("dataset == 'unknown'")

# print data sizes
print("train: "   + str(tr.shape))
print("test: "    + str(ts.shape))
print("known: "   + str(kn.shape))
print("unknown: " + str(un.shape))

train: (2279790, 36)
test: (39821, 36)
known: (2319611, 36)
unknown: (19918, 36)


In [5]:
# List numeric features used as predictors
print(data.columns)
numVars = ["user_ratio_flow", "user_ratio_full", "listen_type", "first_flow",
           "song_plays", "artist_plays", "platform_name1", "platform_name2",
           "song_skips", "artist_skips", "song_session_position", "time_diff"] 

# Create the data input matrix that can be passed to the keras model
tr_data = tr[[column for column in tr.columns if column in numVars]].as_matrix()
ts_data = ts[[column for column in ts.columns if column in numVars]].as_matrix()
kn_data = kn[[column for column in kn.columns if column in numVars]].as_matrix()
un_data = un[[column for column in un.columns if column in numVars]].as_matrix()

Index(['user_id', 'media_id', 'artist_id', 'genre_id', 'context_type',
       'media_duration', 'listen_type', 'user_gender', 'user_age',
       'is_listened', 'sample_id', 'dataset', 'session_id',
       'song_session_position', 'first_flow', 'time_diff', 'hours',
       'is_listened_lag', 'genre_plays', 'genre_skips', 'artist_plays',
       'artist_skips', 'album_plays', 'album_skips', 'song_plays',
       'song_skips', 'user_ratio_flow', 'user_ratio_full', 'genre_ratio',
       'artist_ratio', 'song_ratio', 'platform_name1', 'platform_name2',
       'platform_family1', 'platform_family2', 'row_index'],
      dtype='object')


## 2. MODELING

### 2.1. INITIALIZING

In [6]:
# Create an input layer with embeddings
user_in    = Input(shape = (1,), dtype = 'int64',   name = "user_in")
song_in    = Input(shape = (1,), dtype = 'int64',   name = "song_in")
artist_in  = Input(shape = (1,), dtype = 'int64',   name = "artist_in")
context_in = Input(shape = (1,), dtype = 'int64',   name = "context_in")

# Create an input layer with numeric features
data_in = Input(shape = (tr_data.shape[1],), name = "data_in")

# Counting number of unique ID values
n_users   = tr.user_id.nunique()
n_songs   = tr.media_id.nunique()
n_artists = tr.artist_id.nunique()
n_context = tr.context_type.nunique()

# Create an embedding assigning k latent factors to each ID
u = Embedding(n_users,   100, input_length = 1, embeddings_regularizer = l2(1e-5))(user_in)
s = Embedding(n_songs,   100, input_length = 1, embeddings_regularizer = l2(1e-5))(song_in)
a = Embedding(n_artists, 100, input_length = 1, embeddings_regularizer = l2(1e-5))(artist_in)
c = Embedding(n_context, 100, input_length = 1, embeddings_regularizer = l2(1e-5))(context_in)

# Layer with embeddings
embedding_input = concatenate([a, c])
embedding_input = Flatten()(embedding_input)
embedding_dense = Dense(128, activation = "relu")(embedding_input)
x = Dropout(0.5)(embedding_dense)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x) 
x = Dropout(0.5)(x)
output = Dense(1, activation = "sigmoid")(x)

# Specify the model that we want to use
model = Model([artist_in, context_in], output)
model.compile(optimizer = "Adagrad", loss = "binary_crossentropy", metrics = ['accuracy'])

### 2.2. FIRST STAGE

In [8]:
# run the estimations on training data
model.fit([tr.artist_id, tr.context_type], tr.is_listened, 
validation_data = ([ts.artist_id, ts.context_type], ts.is_listened),
batch_size = int(len(tr)/100), epochs = 10)

Train on 2279790 samples, validate on 39821 samples
Epoch 1/10
2279790/2279790 [==============================] - 87s - loss: 0.6572 - acc: 0.6022 - val_loss: 0.6777 - val_acc: 0.6076
Epoch 2/10
2279790/2279790 [==============================] - 102s - loss: 0.6438 - acc: 0.6143 - val_loss: 0.6690 - val_acc: 0.6076
Epoch 3/10
2279790/2279790 [==============================] - 104s - loss: 0.6391 - acc: 0.6185 - val_loss: 0.6638 - val_acc: 0.6076
Epoch 4/10
2279790/2279790 [==============================] - 97s - loss: 0.6368 - acc: 0.6213 - val_loss: 0.6578 - val_acc: 0.6076
Epoch 5/10
2279790/2279790 [==============================] - 96s - loss: 0.6349 - acc: 0.6228 - val_loss: 0.6538 - val_acc: 0.6096
Epoch 6/10
2279790/2279790 [==============================] - 100s - loss: 0.6336 - acc: 0.6244 - val_loss: 0.6530 - val_acc: 0.6101
Epoch 7/10
2279790/2279790 [==============================] - 109s - loss: 0.6327 - acc: 0.6255 - val_loss: 0.6571 - val_acc: 0.6138
Epoch 8/10
2279790/2

In [9]:
# predict validation data
pred = pd.DataFrame()
pred["row_index"] = ts.row_index
pred["is_listened"] = model.predict([ts.artist_id, ts.context_type])
pred.to_csv(path + "pred_valid/keras_newdata_flow_artist_context_nonum_nouser.csv", index = False)
pred.head(5)

,row_index,is_listened
2167,2167,0.574801
2168,2168,0.652714
2240,2240,0.606595
4314,4314,0.591638
4321,4321,0.688023


In [10]:
# computing AUC
metrics.roc_auc_score(ts.is_listened, pred.is_listened)

0.61134396901850607

### 2.3. SECOND STAGE

In [11]:
# run the estimations on full known data
model.fit([kn.artist_id, kn.context_type], kn.is_listened,
batch_size = int(kn.shape[0]/100), epochs = 10)

Epoch 1/10
2319611/2319611 [==============================] - 89s - loss: 0.6306 - acc: 0.6266    
Epoch 2/10
2319611/2319611 [==============================] - 88s - loss: 0.6302 - acc: 0.6265    
Epoch 3/10
2319611/2319611 [==============================] - 89s - loss: 0.6301 - acc: 0.6259    
Epoch 4/10
2319611/2319611 [==============================] - 90s - loss: 0.6294 - acc: 0.6272    
Epoch 5/10
2319611/2319611 [==============================] - 115s - loss: 0.6292 - acc: 0.6279   
Epoch 6/10
2319611/2319611 [==============================] - 116s - loss: 0.6286 - acc: 0.6281   
Epoch 7/10
2319611/2319611 [==============================] - 119s - loss: 0.6284 - acc: 0.6278   
Epoch 8/10
2319611/2319611 [==============================] - 114s - loss: 0.6280 - acc: 0.6279   
Epoch 9/10
2319611/2319611 [==============================] - 119s - loss: 0.6280 - acc: 0.6278   
Epoch 10/10
2319611/2319611 [==============================] - 115s - loss: 0.6277 - acc: 0.6285   


In [12]:
# predict unknown data
pred = pd.DataFrame()
pred["sample_id"] = un.sample_id.astype(int)
pred["is_listened"] = model.predict([un.artist_id, un.context_type])
pred = pred.sort_values("sample_id")
pred.to_csv(path + "pred_unknown/keras_newdata_flow_artist_context_nonum_nouser.csv", index = False)
pred.head(5)

,sample_id,is_listened
2332218,0,0.613701
2182792,1,0.512030
2092769,2,0.512030
1806368,3,0.512030
2018166,4,0.512030
